In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [ ]:
import nbimporter

Formerly, the cell below was able to ensure that running times were reproducible.  Unfortunately, this does no longer work.  Instead, you have to execute the following command in the terminal before the command `jupyter notebook` is executed:
```
export PYTHONHASHSEED=0
```

In [ ]:
import os
os.environ['PYTHONHASHSEED'] = '0'

# A Backtracking Solver for CSPs

## Utility Functions

The module `ast` provides a parser for Python expressions.

In [ ]:
import ast

The function `collect_variables(expr)` takes a string `expr` that can be interpreted as a Python expression as input and collects all variables occurring in `expr`.  It takes care to remove the function symbols from the names returned by `extract_variables`.

In [ ]:
def collect_variables(expression): 
    tree = ast.parse(expression)
    return { node.id for node in ast.walk(tree) 
                     if  isinstance(node, ast.Name) 
                     if  node.id not in dir(__builtins__)
           }

Below, `dir(__builtins__)` returns a list containing all predefined variables, functions, and classes. 

In [ ]:
dir(__builtins__)

In [ ]:
collect_variables('abs(x - y) + abs(z1 - z2)')

The function `arb(S)` takes a set `S` as input and returns an arbitrary element from 
this set.

In [ ]:
def arb(S):
    "Return some element from the set S."
    for x in S:
        return x

Backtracking is implemented by raising the `Backtrack` exception.  We define this new class of exceptions so that we can distinguish `Backtrack` exceptions from ordinary exceptions.  This is done by creating a new, empty class that is derived from the class `Exception`.  

In [ ]:
class Backtrack(Exception):
    pass

## The Backtracking Solver

The procedure `solve(P)` takes a *constraint satisfaction problem* 
`P` as input.  Here `P` is a triple of the form 
$$ \mathcal{P} = \langle \mathtt{Variables}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where 
- `Variables` is a set of strings which serve as *variables*,
- `Values` is a set of *values* that can be assigned 
  to the variables in the set `Variables`.
- `Constraints` is a set of formulas from first order logic.  
  
The second argument `debug` controlls the printing of debugging information.  If set to true, every partial assignment that is created during the search, will be printed.
  
The main purpose of the function `solve` is to convert the CSP `P` into an 
*augmented CSP* where every constraint $f$ is annotated with
the variables ocurring in $f$.  This annotated CSP is then solved using the function
`backtrack_search`.

In [ ]:
def solve(P, debug=False):
    Variables, Values, Constraints = P
    csp = (Variables, Values, [(f, collect_variables(f)) for f in Constraints])
    try:
        return backtrack_search({}, csp, debug)
    except Backtrack:
        return None

The function `backtrack_search` takes two arguments:
- `Assignment` is a partial variable assignment that is represented as a dictionary.  Initially, this assignment will be the empty dictionary.  Every recursive call of `backtrack_search` adds the assignment of one variable to the given assignment.  The important invariant of recursive calls of `backtrack_search` is that `Assignment` is *consistent*, i.e. all constraints $f$ that contain only variables from the set $\mathtt{dom}(\mathtt{Assignment})$ are satisfied.
- `P` is an *augmented constraint satisfaction problem*, 
   i.e. `P` is a triple of the form 
   $$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
    where 
    - $\mathtt{Vars}$ is a set of strings which serve as *variables*,
    - $\mathtt{Values}$ is a set of *values* that can be assigned 
      to the variables in $\mathtt{Vars}$.
    - $\mathtt{Constraints}$ is a set of pairs of the form $(f, V)$ where $f$ is a Boolean Python 
      expression, while $V$ is the set of variables occuring in $f$.
- `debug` is a Boolean flag.  If this flag is true, every partial assignment is printed.      
The function `backtrack_search` tries to find a solution of `P` by recursively augmenting `Assignment`.

Normally, the `Variables` stored in the CSP are represented as a set.  However, for didactical purposes it is also possible to store the variables as a list.  This way, the order in which variables are chosen can be controlled.

In [ ]:
def backtrack_search(Assignment, P, debug):
    if debug:
        print(Assignment)
    Variables, Values, Constraints = P
    if len(Assignment) == len(Variables):
        return Assignment
    if isinstance(Variables, set):
        var = arb(Variables - Assignment.keys())
    else: # if Variables is a list we choose the first unassigned variable
        var = [x for x in Variables if x not in Assignment][0]
    for value in Values:
        try:
            if is_consistent(var, value, Assignment, Constraints):
                NewAss = Assignment.copy()
                NewAss[var] = value
                return backtrack_search(NewAss, P, debug)
        except Backtrack:
            continue
    raise Backtrack()  

The function $\texttt{is_consistent}(\texttt{var}, \texttt{value}, \texttt{Assignment}, \texttt{Constraints})$ takes four arguments:
- `var` is a variable that does not occur in $\texttt{Assignment}$,
- `value` is a value that can be substituted for this variable,
- `Assignment` is a *consistent* partial variable assignment. 
- `Constraints` is a set of pairs of the form $\langle f, V \rangle$ where $f$ is a formula and $V$ is the set of variables occurring in $f$.

This function returns `True` iff the partial variable assignment 
$$\texttt{Assignment} \cup \bigl\{\langle\texttt{var} \mapsto\texttt{value}\rangle\bigr\}$$
is consistent with all the constraints $f$ occurring in `Constraints`.

**Note** that the function `eval` mutates the given variable assignment.  Therefore it is necessary to copy the given `Assignment`.  

In [ ]:
def is_consistent(var, value, Assignment, Constraints):
    NewAssign      = Assignment.copy()
    NewAssign[var] = value
    return all(eval(f, NewAssign) for (f, Vs) in Constraints
                                  if  var in Vs and Vs <= NewAssign.keys()
              )

# Map Coloring

In [ ]:
from MapColoring import map_coloring_csp

In [ ]:
P = map_coloring_csp()
P

In [ ]:
%%time
Solution = solve(P, True)

If we reorder the variables appropriately, the solver never needs to backtrack.

In [ ]:
P2 = (['SA', 'WA',  'NT', 'Q', 'NSW', 'V', 'T'], P[1], P[2])

In [ ]:
%%time
Solution = solve(P2, True)

## Solving the Eight-Queens-Puzzle

In [ ]:
from NQueensProblemCSP import create_csp, show_solution

In [ ]:
P = create_csp(8)

Backtracking search takes about 4 milliseconds on my iMac to solve the eight queens puzzle.

In [ ]:
%%time
Solution = solve(P, debug=True)
print(f'Solution = {Solution}')

In [ ]:
show_solution(Solution)

Backtracking is able to solve the $16$ queens problem in about $2.7$ seconds.

In [ ]:
P = create_csp(16)

In [ ]:
%%time
Solution = solve(P)
print(f'Solution = {Solution}')

## Solving the *Zebra Puzzle*

In [ ]:
from Zebra import zebra_csp, show_solution

In [ ]:
zebra = zebra_csp()

Backtracking takes about $30$ seconds to solve the [Zebra Puzzle](https://en.wikipedia.org/wiki/Zebra_Puzzle).
We develop a better algorithm soon.

In [ ]:
%%time
Solution = solve(zebra)

In [ ]:
show_solution(Solution)

## Solving the Cypto-Arithmetic Puzzle

In [ ]:
from CryptoArithmetic import crypto_csp, crypto_csp_hard, show_solution

In [ ]:
csp = crypto_csp()
csp

It takes about 31 seconds to solve the crypto-arithmetic puzzle.

In [ ]:
%%time
Solution = solve(csp)

In [ ]:
show_solution(Solution)

Let's try the harder version.

In [ ]:
csp = crypto_csp_hard()

In [ ]:
%%time
Solution = solve(csp)

It takes about 3 minutes and 30 seconds to solve the hard version.